# Markowitz Porftolio Theory

**Prerequisites**

- Convex Optimization: Quadratic Programming

**Outcomes**

- Understand core terminology of investment portfolios
- Solve the standard MPT optimization problem using QP
- Understand the risk/return tradeoff
- Compute and plot the *efficient frontier*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

plt.style.use(["ggplot"])
pd.set_option('display.float_format', lambda x: '%.5f' % x)

prices = pd.read_csv("equities.csv", parse_dates=["Date"], index_col=["Date"])
prices_jan15 = prices.loc["2015-01"]

prices.iloc[[0, 1, 2, -3, -2, -1], :]

## Background: Definitions, Notation, and Moments

- Let $i = 1, \dots, N$ index assets and $t$ index time
- Let $p_{i,t}$ be the price of asset $i$ on date $t$
- A **portfolio** is a collection of assets
    - Can be represented by (potentially time varying) *shares*: $n_{i,t}$
- Value of a portoflio is $$V_t = \sum_{i} n_{i,t} p_{i,t}$$
- We might also represent positions by asset *weights* $$w_{i,t} \equiv \frac{n_{i,t} p_{i,t}}{V_t}$$

### Example Portfolio

Let's track a sample portfolio over the month of January 2015

Suppose we start out holding one share of each of the stocks in the `prices_jan15` DataFrame

On Jan. 12 we sell our share of AAPL and use proceeds to buy NKE

On Jan. 20 we sell our 1/2 our DIS to and buy AAPL

On Jan. 28 we sell the remaining 1/2 of DIS to buy MSFT

The code below creates a DataFrame `portfolio` that tracks our positions in each of the assets over January 2015

In [ ]:
portfolio = pd.DataFrame(np.ones(prices_jan15.shape), index=prices_jan15.index, columns=list(prices_jan15))
transactions = [
    dict(date="2015-01-12", sell="AAPL", frac=1, buy="NKE"),
    dict(date="2015-01-20", sell="DIS", frac=1/2, buy="AAPL"),
    dict(date="2015-01-28", sell="DIS", frac=1, buy="MSFT")
]

for tx in transactions:
    new_val = prices_jan15 * portfolio
    # get sale value
    dollars = new_val.at[tx["date"], tx["sell"]] * tx["frac"]
    
    # use proceeds to purchase new asset
    portfolio.loc[tx["date"]:, tx["buy"]] += dollars / prices.at[tx["date"], tx["buy"]]
    
    # lower holdings of asset we sold
    portfolio.loc[tx["date"]:, tx["sell"]] *= (1 - tx["frac"])

We can look at the porfolio holdings over time

In [ ]:
portfolio

Track the value of each position over time

In [ ]:
position_value = portfolio * prices_jan15
position_value

If we sum across each row we can compute the value of the portfolio at each date

In [ ]:
portfolio_value = position_value.sum(axis=1)
portfolio_value.plot(figsize=(8, 4))

Finally, we can compute the portfolio weights

In [ ]:
portfolio_weights = position_value.div(portfolio_value, axis=0)  # n_{it} p_{it} / V_t
portfolio_weights

In [ ]:
portfolio_weights.plot(figsize=(10, 6))

### Asset Returns

We will assume that each position is held for exactly one month

Let $p$ be the price at which we enter the position, $p'$ be the price at which we exit, and $n$ be the size of our position

The return of the position is $$r \equiv \frac{p' - p}{p}$$

We can compute monthly returns of our `prices` DataFrame using the `pct_change` method

In [ ]:
monthly_r = prices.resample("BM").first().pct_change()

fig, ax = plt.subplots(2, 1, figsize=(10, 6))
prices.plot(lw=1, ax=ax[0])
monthly_r.plot(figsize=(10, 6), lw=1, ax=ax[1], legend=False)
fig.tight_layout();

### Portfolio Returns

Let $w$ be a weight vector defining a portfolio

The return on the portfolio $r_p$ is a weighted average of the asset returns: $$r_p \equiv \sum_i w_i r_i$$

We can compute the daily returns of our January 2015 portfolio:

In [ ]:
daily_r_jan15 = prices_jan15.pct_change()
r_p_jan15 = (portfolio_weights * daily_r_jan15).sum(axis=1).iloc[1:]
r_p_jan15.plot()

### Moments of Returns

Given our time series of returns for each asset, we can compute various moments

Two moments of particular interest for us today are

1. $\mu$: Average returns for each asset
2. $\Sigma$: Covariance of returns across assets

Given our monthly returns DataFrame we can compute these moments using the `.mean` and `.cov` methods:

In [ ]:
mu = monthly_r.mean()
mu

In [ ]:
Sigma = monthly_r.cov()
Sigma

### Porfolio Moments

We can also compute moments for a portfolio defined by a weight vector $w$

Given average returns for assets in the portfolio ($\mu$) and the covariance of asset returns ($\Sigma$), the portfolio return and variance are given by

\begin{align*}
\mu_p &= w^T \mu \\
\sigma_p &= w^T \Sigma w
\end{align*}

Note that $\mu_p$ and $\sigma_p$ are both scalars, not a vector and Matrix like $\mu$ and $\Sigma$

## Portfolio Optimization

Portfolio optimization is the task of selecting the "best" portfolio from the set of all feasible portfolios

A simplistic notion of "best" is to put all weight on the single asset we believe will have largest percent increase in price

If our beliefs are correct, this portfolio would maximize returns

However, this portfolio would be fully exposed to the price movements of a single asset

This is risky business!

Portfolios provide *diversification*, which can lower the risk just mentioned

There are theories of portfolio optimization that allow the investor to balance the risk/return tradeoff

We will study the foundational portfolio theory called Markowitz portfolio theory (MPT)

> Another common name for Markowitz' theory is *modern portfolio theory*, which fortunately has the same acronym: MPT

## Markowitz Portfolio Optimization

The MPT framework assumes that investors are risk adverse

This means that if two different portfolios have the same expected return, investors will prefer the one with lower risk

Risk is summarized by the variance of portfolio returns

### MPT Problem

The Markowitz' portfolio optimization problem takes both expected returns and expected risk into account

Here is the form of the QP:

\begin{align*}
\min_{w} \quad & \gamma w^T \Sigma w  - \mu^T w \\
s.t. \quad & \mathbf{1}^T w = 1
\end{align*}

where $\gamma$ is a constant that defines risk sensitivity



The MPT problem can be solved with `cvxpy` as follows

> Note we will add the additional constraint that $w >= 0$, which means that we do not allow short selling and consider long-only portfolios

In [ ]:
import cvxpy as cp

def solve_mpt(μ, Σ, gamma, labels=None):
    w = cp.Variable(len(μ))
    obj = gamma * cp.quad_form(w, Σ) - μ @ w
    prob = cp.Problem(
        cp.Minimize(obj),
        [sum(w) == 1, w >= 0]
    )
    prob.solve()
        
    w_out = w.value
    if labels is not None:
        w_out = pd.Series(w_out, index=labels)
    return w_out, prob

Let's use our equity data to formulate and solve the MPT problem

We will use the data from January 2014 through December 2016 to solve the the Markowitz' problem 

In [ ]:
returns = prices.pct_change()
mu_pd = returns.loc[:"2016", :].mean()
Sigma_pd = returns.loc[:"2016", :].cov()
order = list(Sigma_pd)

# user order1 to make sure data is aligned
mu = mu_pd.loc[order].to_numpy()
Sigma = Sigma_pd.to_numpy()

w, prob = solve_mpt(mu, Sigma, gamma=1.0, labels=order)
w 

Now let's evaluate the performance of the portfolio `w` using the data for 2017

We'll make the assumption that we start 2017 with $10,000 of capital and see how much we end up with!

In [ ]:
capital = 10000
prices_2017 = prices.loc["2017", :]

# split capital across assets according to shares
dollars_per_asset = w * capital

# Purchase portfolio: $budget / ($ per share) = shares
shares = dollars_per_asset / prices_2017.iloc[0, :]

# record shares
portfolio = pd.DataFrame(index=prices_2017.index, columns=shares.index)
portfolio.loc[prices_2017.index, w.index] = shares.to_numpy()

We now track the value of each position

In [ ]:
V17 = (prices_2017 * portfolio)
V17

And the value of the portfolio

In [ ]:
V17_p = V17.sum(axis=1)
V17_p.plot(figsize=(8, 4));

This would have been a great investment!

The 2017 return on each position is

In [ ]:
(V17.iloc[-1] - V17.iloc[0]) / V17.iloc[0]

In [ ]:
w

This is surprising! Even though we had zero shares of AAPL, DIS, MCD, and NKE we are seeing positive returns on the year

How is this possible?

Let's look at first and last value of `V17` for DIS

In [ ]:
V17.iloc[[0, -1]]["DIS"].to_numpy()

These are both effectively zero

However, when we divide by `-5.025378273933108e-19` we might end up with just about anything because this number is so close to zero

In fact, the maximum resolution for a 64-bit float (the default for Python's `float` on a 64-bit operating system) is on the order of 1e-16

This means that for numbers whose difference is less than 1e-16, there are no guarantees about what the computer will compute!

To overcome this numerical issue we will set V17 equal to a small number (1e-6) for all rows in columns that have near zero weight in our portfolio

By using the *same* small number, we will compute zero where we are supposed to compute zero

Then, we'll recompute the yearly returns computation

In [ ]:
V17_adj = V17.copy()
zeros = abs(w) < 1e-6
V17_adj.loc[:, zeros] = 1e-6

(V17_adj.iloc[-1, :] - V17_adj.iloc[0, :]) / V17_adj.iloc[0, :]

That makes more sense!

Let's calculate yearly returns for the portfolio

In [ ]:
r17_p = (V17_p.iloc[-1] - V17_p.iloc[0]) / V17_p.iloc[0]
r17_p

This is a great return

Each dollar we invested to start the year turned into $1.42 by the end of the year

Let's now compute the first two moments of the returns for each position and the portfolio

In [ ]:
r17 = V17_adj.pct_change()
mu17 = r17.mean()
mu17

In [ ]:
sigma17 = r17.var()
sigma17

And for the portfolio

In [ ]:
r17_p = V17_p.pct_change()
mu17_p = r17_p.mean()
mu17_p

In [ ]:
sigma17_p = r17_p.var()
sigma17_p

Notice that the average returns of the portfolio are very similar to the average returns of each asset

In [ ]:
(mu17_p - mu17)  / mu17

However, the variance of portfolio is significantly lower than the variance of either asset

In [ ]:
(sigma17_p - sigma17)  / sigma17

This illustrates how diversification through a portfolio can lower the risk of investment

### Other Portfolios: The Role of $\gamma$

Above we showed computations for a single portfolio, corresponding to the MPT portoflio for $\gamma = 1$

There are an infinite number of other portfolios

We can compute some of them by varying $\gamma$

Below we use the `np.logspace` routine to construct a grid of values of $\gamma$

The spacing between points will be constant on the log base 10 scale

In [ ]:
gammas = np.logspace(-4, 4, 50)
gammas

Next, we will solve the MPT problem for these different levels of gamma

In [ ]:
gamma_parts = []

for gamma in gammas:
    w_gamma, _ = solve_mpt(mu, Sigma, gamma=gamma, labels=order)
    w_gamma["gamma"] = gamma
    gamma_parts.append(w_gamma)

w_gammas = pd.concat(gamma_parts, axis=1).T

In [ ]:
w_gammas.iloc[::5, :]

Notice that for very small values of $\gamma$, the solution to the MPT problem puts all weight on MSFT

Why?

Because MSFT has the highest expected return:

In [ ]:
mu_pd

This happens because with $\gamma$ near zero, the MPT problem reduces to the *linear program*

\begin{align*}
\min_w \quad & - \mu^T w \\
s.t. \quad & \mathbf{1}^T w = 1 \\
& w >= 0
\end{align*}

The fact that we end up with a portfolio with 100% weight on MSFT is a manifestation of the *corner solution problem* of linear programming

In terms of our application, the MPT problem reduces to putting all capital into the single asset we believe will have the highest return

Let's show those same rows of `w_gammas` again and watch what happens as $\gamma$ increases

In [ ]:
w_gammas.iloc[::5, :]

Notice that the largest $\gamma$ shown above has a portfolio with non-zero weight on all 6 assets

The highest share in our portfolio belongs to MCD

Why?

To answer this question we need to look at both the expected returns and the expected covariance

In [ ]:
mu_pd

In [ ]:
Sigma_pd

Although MCD had the lowest expected return, it also has the lowest variance

Also, when considering covariance, notice the general trend that the covariance between any asset and MCD is typically lower than the covariance between that asset and any other asset

If we compute the sum of covariances for each asset, we'll see that the average covariance is lowest for MCD

In [ ]:
Sigma_pd.mean()

How does this translate into the MPT optimal portfolio?

Because MCD has a low covariance with other assets, the MPT portfolio with a high $\gamma$ really likes MCD

However, because expected returns for MCD are the lowest the optimizer wants to avoid MCD

These two forces are balanced, with $\gamma$ determining the relative significance of each component

### The Sharpe Ratio

Above we used QP to solve for multiple portfolios that solve the MPT problem for various levels of the risk aversion parameter $\gamma$

If we are starting 2017 and asked to pick one of these portfolios to use for the year, how would we choose?

We determine to be disciplined and compute a single number that summarizes our beliefs about the benefits of each portfolio

We'll call this number the performance metric

The portoflio we select will be the one that maximizes our performance metric

One option for the performance metric is the expected return on the portfolio

If we think carefully about this, we will realize that this will lead us to portfolio associated with the smallest value of $\gamma$

This portoflio put 100% weight on MSFT

This will remove diversification and cause us to take on excess risk

We would like to use a performance metric that takes into account both the expected return and expected variance (or risk) of the portfolio

One such metric is the Sharpe ratio, which is defined as

$$S = \frac{E[R_p - R_f]}{E[\sigma_p]}$$

Where $R_f$ is the risk free rate -- or the gross return that can be achieved with zero risk

For our experiements in this lecture we will assume we don't have access to a risk free investent opportunity and will compute $$S = \frac{E[R_p]}{E[\sigma_p]}$$

We can compute the Sharpe ratio for each of the portfolios in our `w_gammas` DataFrame

In [ ]:
def sharpe_ratio(w, mu, Sigma):
    r_p = w.T @ mu
    sigma_p = np.sqrt(w.T @ Sigma @ w)
    sharpe = r_p / sigma_p
    return pd.Series(dict(r=r_p, sigma=sigma_p, sharpe=sharpe))

In [ ]:
w_gammas_hat = w_gammas.drop(["gamma"], axis=1)
sharpes = w_gammas_hat.apply(sharpe_ratio, args=(mu_pd, Sigma_pd), axis=1)
sharpes["gamma"] = w_gammas["gamma"]
sharpes.iloc[::5, :]

Notice a few patterns as $\gamma$ increases:

- r decreases
- sigma decreases
- the sharpe ratio initially increases, but then begins to decrease


This pattern is extra evident in a chart

In [ ]:
ax = sharpes.plot.scatter(x="sigma", y="r", c=sharpes["sharpe"], cmap="plasma", figsize=(8,6));
ax.set_xlabel("sigma");

### The Efficient Frontier

In our original formulation of the MPT optimization problem from above, $\gamma$ played the role of risk sensitivity

We saw that as $\gamma$ increased, the optimizer found a portfolio that was more conservative at the expense of lower expected returns

There is an alternative formulation of the MPT problem that seeks to find the minimum variance portfolio that achieves a specified expected return

This alternative represenation of the MPT problem is

\begin{align*}
\min_w \quad & w^T \Sigma w \\
s.t. \quad & \mathbf{1}^T w = 1 \\
& \mu^T w = r^*,
\end{align*}

where $r^*$ is the desired expected return

By solving this alternative MPT formulation, we can pick the "y-coordinate" in the (sigma, r) plot from above

Let's give it a shot (continuing to add the no-short condition)!

In [ ]:
def solve_mpt_r(μ, Σ, rstar, labels):
    w = cp.Variable(len(μ))
    
    obj = cp.quad_form(w, Σ)
    prob = cp.Problem(
        cp.Minimize(obj),
        [sum(w) == 1, w >= 0, μ @ w == rstar]
    )
    
    prob.solve()
    
    out_w = pd.Series(w.value, index=labels)
    return out_w, prob    

In [ ]:
rgrid = np.linspace(sharpes["r"].min() * 0.95, sharpes["r"].max(), 100)

rstar_parts = []

for rstar in rgrid:
    w_rstar, prob_rstar = solve_mpt_r(mu, Sigma, rstar=rstar, labels=order)
    w_rstar["rstar"] = rstar
    w_rstar["sigma"] = np.sqrt(prob_rstar.value)
    rstar_parts.append(w_rstar)

w_rstars = pd.concat(rstar_parts, axis=1).T


In [ ]:
fig, ax = plt.subplots()
ax.plot(w_rstars["sigma"], rgrid)
ax.set_ylabel("r")
ax.set_xlabel("sigma");

The plot above is called the *efficient frontier*

For each value portfolio returns $r$, the frontier represents minimum possible portfolio variance that achieves that return, subject to our beliefs as encoded in $\mu$ and $\Sigma$

We can see this bounding feature of the efficient frontier more clearly if we simulate random portfolio weights and plot them alongside the frontier

In [ ]:
rand_parts = []

for _ in range(1500):
    w = np.random.rand(6)
    w /= w.sum()
    rand_parts.append(sharpe_ratio(w, mu_pd, Sigma_pd))


rand_sharpes = pd.concat(rand_parts, axis=1).T

In [ ]:
ax = rand_sharpes.plot.scatter(x="sigma", y="r", c=rand_sharpes["sharpe"], cmap="plasma", figsize=(8, 5))
ax.plot(w_rstars["sigma"], rgrid)
ax.set_title("Efficient Frontier")
ax.set_xlabel("sigma");
